# Task 1 f, Calculating reserve factors against column buckling using Euler-Johnson

## We need from the data file:
- volume of the element
- Iyy calculated with `height_str, width_str, thickness_web, thickness_flange, thickness_skin, stringer_pitch`
- dimensions
- `EulerJohnson(EModulus, I_y, area, length, height_str, thickness_flange, thickness_web, radius, sigma_yield, sigma_applied, c=1)`

## Please enter your name (daniel, fabian, felix, yannis)

In [1]:
name = 'fabian'

## Imports

In [2]:
import pandas as pd
import numpy as np
import sys 
import os
sys.path.insert(0, os.path.abspath('..'))
sys.path.insert(0, os.path.abspath('../formulas'))

from formulas.columnbuckling import *
from formulas.panels import *
from formulas.strength import *
from formulas.helpers import *

## Constants

In [3]:
personal_data = personal_data_provider(name)
sigma_yield = personal_data[0]
EModulus = personal_data[1]
print(f"Your personal data is: sigma_yield = {sigma_yield}, EModulus = {EModulus}. Please verify.")

stringer_pitch = 200
effective_width = stringer_pitch/2

panel_element_length = 750/3
stringer_element_length = 750/3

# Hat stringer parameters
DIM1 = 25
DIM2 = 2
DIM3 = 20
DIM4 = 10

thickness_skin = 10


Your personal data is: sigma_yield = 490, EModulus = 65420.46. Please verify.


## Cross-Section Properties of the hat stringer

In [4]:
cs_properties = crosssectional_properties_hat_skin(DIM1, DIM2, DIM3, DIM4, thickness_skin, effective_width, stringer_depth=stringer_element_length)
effective_panel_volume = effective_width * thickness_skin * panel_element_length # 750 mm is the length of the panel, 3 FE per panel

## Open stress values & properties of panels

In [5]:
#Imoport panel stress file and reformat 
paneldf = pd.read_excel(f'../data/{name}/panel.xlsx', index_col=0)
paneldf = paneldf.drop(['sigmaZZ', 'sigmaXZ', 'sigmaYZ'],axis=1)
paneldf = paneldf.reset_index()

# Import stringer properties
stringerdf = pd.read_excel(f'../data/{name}/stringer.xlsx', index_col=0)
stringerdf = stringerdf.reset_index()
print(stringerdf.head(5))

#Import panel properties and reformat
panelPropertiesdf = pd.read_excel(f'../data/{name}/panel_properties.xlsx', index_col=0)
panelPropertiesdf = panelPropertiesdf.reset_index()
panelPropertiesdf = panelPropertiesdf.rename(columns={'elements': 'Element ID'})

print(paneldf.head(5))
print(panelPropertiesdf.head(5))

   Element ID Component Name  sigmaXX        Load Case
0          40      stringer1   -77.67  Subcase 1 (LC1)
1          41      stringer1   -70.60  Subcase 1 (LC1)
2          42      stringer1   -79.68  Subcase 1 (LC1)
3          43      stringer2   -81.30  Subcase 1 (LC1)
4          44      stringer2   -85.24  Subcase 1 (LC1)
   Element ID Component Name  sigmaXX  sigmaYY  sigmaXY        Load Case
0           1         panel1   -66.67   12.750    26.25  Subcase 1 (LC1)
1           2         panel1   -82.97   20.870    27.10  Subcase 1 (LC1)
2           3         panel1   -67.63   20.740    36.97  Subcase 1 (LC1)
3           4         panel2   -97.67    2.415    34.52  Subcase 1 (LC1)
4           5         panel2   -89.50   25.350    27.04  Subcase 1 (LC1)
   Element ID     mass  thickness
0           1  0.00054          4
1           2  0.00054          4
2           3  0.00054          4
3           4  0.00054          4
4           5  0.00054          4


## Split up the loadcases

In [6]:
loadCase1df = paneldf[paneldf['Load Case'] == 'Subcase 1 (LC1)']
loadCase2df = paneldf[paneldf['Load Case'] == 'Subcase 2 (LC2)']
loadCase3df = paneldf[paneldf['Load Case'] == 'Subcase 3 (LC3)']
# Add the data to load case 
loadCase1df = pd.merge(loadCase1df, panelPropertiesdf, on='Element ID')
loadCase2df = pd.merge(loadCase2df, panelPropertiesdf, on='Element ID')
loadCase3df = pd.merge(loadCase3df, panelPropertiesdf, on='Element ID')

# Now for the stringer properties, split into load cases
stringer_loadCase1d = stringerdf[stringerdf['Load Case'] == 'Subcase 1 (LC1)']
stringer_loadCase2d = stringerdf[stringerdf['Load Case'] == 'Subcase 2 (LC2)']
stringer_loadCase3d = stringerdf[stringerdf['Load Case'] == 'Subcase 3 (LC3)']
print(stringer_loadCase1d.head(5))

#Give some overview 
loadCase1df.head(5)


   Element ID Component Name  sigmaXX        Load Case
0          40      stringer1   -77.67  Subcase 1 (LC1)
1          41      stringer1   -70.60  Subcase 1 (LC1)
2          42      stringer1   -79.68  Subcase 1 (LC1)
3          43      stringer2   -81.30  Subcase 1 (LC1)
4          44      stringer2   -85.24  Subcase 1 (LC1)


,Element ID,Component Name,sigmaXX,sigmaYY,sigmaXY,Load Case,mass,thickness
0,1,panel1,-66.67,12.750,26.25,Subcase 1 (LC1),0.00054,4
1,2,panel1,-82.97,20.870,27.10,Subcase 1 (LC1),0.00054,4
2,3,panel1,-67.63,20.740,36.97,Subcase 1 (LC1),0.00054,4
3,4,panel2,-97.67,2.415,34.52,Subcase 1 (LC1),0.00054,4
4,5,panel2,-89.50,25.350,27.04,Subcase 1 (LC1),0.00054,4


## Combine stringers with their adjacent panel-halfs

In [7]:
cs_properties = crosssectional_properties_hat_skin(DIM1, DIM2, DIM3, DIM4, thickness_skin, effective_width, stringer_depth=stringer_element_length)
Iyy = cs_properties[0]
A_tot = cs_properties[1]